In [1]:
import os
from PIL import Image
import time
import json
import yaml

In [66]:
# Image Resize (width: 640): 너무 작다.

def resize_image_to_width_640(filename):
    with Image.open(filename) as img:
        width, height = img.size
        ratio = height / width
        
        new_width = 640
        new_height = int(ratio * new_width)

        # Image.Dither.None 옵션을 사용하여 속도 약 2배 빨라짐
        im_resize = img.resize((new_width, new_height), Image.Dither.NONE)

    os.remove(filename)
    im_resize.save(filename)

In [69]:
# Image Resize (height: 640)

def resize_image_to_height_640(filename):
    with Image.open(filename) as img:
        width, height = img.size
        ratio = width / height
        
        new_height = 640
        new_width = int(ratio * new_height)

        # Image.Dither.None 옵션을 사용하여 속도 약 2배 빨라짐
        im_resize = img.resize((new_width, new_height), Image.Dither.NONE)

    os.remove(filename)
    im_resize.save(filename)

In [70]:
# Images Resize by recursive

def resize_images_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            resize_images_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.jpg':
                    resize_image_to_height_640(os.path.join(src, filename))

In [72]:
# Resize Train Data

start = time.time()
resize_images_recursive('./data/train/원천데이터')
print("time :", time.time() - start)

time : 7316.926134109497


In [71]:
# Resize Validation Data

start = time.time()
resize_images_recursive('./data/validation/원천데이터')
print("time :", time.time() - start)

time : 910.5802710056305


In [ ]:
all_parts = { "P00":"차량전체", "P01":"프론트범퍼", "P02":"리어범퍼", "P03":"타이어(휠)", "P04":"A필러",
       "P05":"C필러", "P06":"사이드미러", "P07":"앞도어", "P08":"뒷도어", "P09":"라디에이터그릴",
       "P10":"헤드램프", "P11":"리어램프", "P12":"보닛", "P13":"트렁크", "P14":"루프", }

In [2]:
ignore_parts = ["P01", "P02", "P03", "P04", "P05", "P06", "P07", "P08", "P10", "P11", "P12", "P13", "P14"]

In [3]:
# Get Classes List

model_classes = []

def get_model_classes(src):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
        
    for obj in jess_dict['learningDataInfo']['objects']:
        classID = obj['classId'].split('.')[0]
        if classID not in ignore_parts:
            rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:15]
            if rawDataID not in model_classes:
                model_classes.append(rawDataID)
            break

In [4]:
# Get Classes List by recursive

def get_classes_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            get_classes_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.json':
                    get_model_classes(os.path.join(src, filename))

In [5]:
# Get Classes

start = time.time()
get_classes_recursive('./YOLO_DATA_P00/data/train/labels')
print("time :", time.time() - start)

time : 61.993377923965454


In [6]:
model_classes[0]

'KI_056'

In [7]:
sorted_model_classes = sorted(model_classes)
sorted_model_classes[0]

'HY_025'

In [8]:
len(sorted_model_classes)

32

In [9]:
# Make Label with parts

def make_label_with_parts(src):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
    
    rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:15]
    resolution = jess_dict['rawDataInfo']['resolution']
    width, height = int(resolution.split('*')[0]), int(resolution.split('*')[1])
    
    used_class_id = 0
    with open(os.path.splitext(src)[0]+'.txt', 'w') as f:
        for obj in jess_dict['learningDataInfo']['objects']:
            classID = obj['classId'].split('.')[0]
            if classID not in ignore_parts:
                used_class_id += 1
                cx = (int(obj['coords']['tl']['x']) + int(obj['coords']['tr']['x'])) / 2 / width
                cy = (int(obj['coords']['tl']['y']) + int(obj['coords']['bl']['y'])) / 2 / height
                w = (int(obj['coords']['tr']['x']) - int(obj['coords']['tl']['x'])) / width
                h = (int(obj['coords']['bl']['y']) - int(obj['coords']['tl']['y'])) / height

                txt = str(sorted_model_classes.index(rawDataID)) + ' ' + str(cx) + ' ' + str(cy) + ' ' + str(w) + ' ' + str(h) + '\n'

                f.write(txt)    
                
    if used_class_id == 0:
        os.remove(os.path.splitext(src)[0]+'.txt')
        img_path = os.path.splitext(src)[0].replace("labels", "images")
        os.remove(img_path+'.jpg')
    
    os.remove(src)

In [10]:
# Make Labels with parts by recursive

def make_labels_with_parts_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            make_labels_with_parts_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.json':
                    make_label_with_parts(os.path.join(src, filename))

In [12]:
# Make Labels for train

start = time.time()
make_labels_with_parts_recursive('./YOLO_DATA_P00/data/train/labels')
print("time :", time.time() - start)

time : 139.82278180122375


In [11]:
# Make Labels for validation

start = time.time()
make_labels_with_parts_recursive('./YOLO_DATA_P00/data/validation/labels')
print("time :", time.time() - start)

time : 20.765110969543457


In [13]:
# Make dict from sorted_model_classes

dict_classes = {i : c for i, c in enumerate(sorted_model_classes)}

In [14]:
# Make yaml file for YOLOv5

yaml_file = './YOLO_DATA_P00/data/car_model_P00.yaml'

yaml_data = dict(
    path = "YOLO_DATA_P00/data",
    train = "train",
    val = "validation",
    nc = len(dict_classes),
    names = dict_classes
)

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f, explicit_start = True, default_flow_style = False)

In [15]:
dict_classes

{0: 'HY_025',
 1: 'HY_041',
 2: 'HY_042',
 3: 'HY_050',
 4: 'HY_053',
 5: 'HY_054',
 6: 'HY_058',
 7: 'HY_063',
 8: 'HY_064',
 9: 'HY_067',
 10: 'HY_069',
 11: 'HY_073',
 12: 'HY_083',
 13: 'HY_091',
 14: 'HY_098',
 15: 'HY_099',
 16: 'KI_026',
 17: 'KI_027',
 18: 'KI_028',
 19: 'KI_029',
 20: 'KI_043',
 21: 'KI_047',
 22: 'KI_051',
 23: 'KI_052',
 24: 'KI_056',
 25: 'KI_057',
 26: 'KI_059',
 27: 'KI_060',
 28: 'KI_062',
 29: 'KI_065',
 30: 'KI_066',
 31: 'KI_079'}